In [1]:
# Install the Snowflake connector if not already installed:
# !pip install snowflake-connector-python

import snowflake.connector
import pandas as pd
import numpy as np

# Connect to Snowflake using given credentials
conn = snowflake.connector.connect(
    user='MUDIT',
    password='Testing@123123',
    account='BCEMHHI-LB94703',
    warehouse='COMPUTE_WH',
    database='JOB_RECOMMENDATIONS',
    schema='JOB_DATA',
    role='ACCOUNTADMIN'
)
print("Connected to Snowflake successfully.")


Connected to Snowflake successfully.


In [2]:
# Execute a query to retrieve all records from the JOBS table
query = "SELECT * FROM JOB_DESC;"
cur = conn.cursor()
cur.execute(query)

# Fetch all results into a pandas DataFrame
df_jobs = cur.fetch_pandas_all()  # Loads all rows into a DataFrame&#8203;:contentReference[oaicite:1]{index=1}
print(f"Retrieved {len(df_jobs)} job postings.")
df_jobs.head(3)  # display first few rows for verification (optional)


Retrieved 1615940 job postings.


,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,2600342200917599,None,None,$61K-$106K,"Capitol Hill, Saipan",None,NaN,NaN,None,NaN,...,None,Purchasing Agent,Inventory Manager,None,An Inventory Manager oversees inventory levels...,None,Inventory control Demand forecasting Supply ch...,None,Kyndryl Holdings,None
1,1097571695278272,None,None,$57K-$86K,Banjul,None,NaN,NaN,None,NaN,...,None,Graphic Designer,Web Graphic Designer,None,Web Graphic Designers create visually appealin...,None,"Graphic design tools (e.g., Adobe Creative Sui...",None,Ambuja Cements,None
2,393705790719989,None,None,$60K-$103K,Tashkent,None,NaN,NaN,None,NaN,...,None,Physician Assistant,Surgical Physician Assistant,None,"Assist surgeons in the operating room, perform...",None,Surgical procedures and techniques Operating r...,None,Whitehaven Coal,None


In [3]:
import re

# Text cleaning function: lowercase, remove punctuation/newlines, normalize spaces&#8203;:contentReference[oaicite:4]{index=4}
def clean_text(text):
    if not isinstance(text, str):
        text = str(text) if text is not None else ""
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and newlines (anything not alphanumeric or whitespace)
    text = re.sub(r'[^\w\s]', ' ', text)  # replace punctuation with space&#8203;:contentReference[oaicite:5]{index=5}
    text = re.sub(r'\s+', ' ', text)      # collapse multiple spaces/newlines into one&#8203;:contentReference[oaicite:6]{index=6}
    return text.strip()

# Define field weights based on importance&#8203;:contentReference[oaicite:7]{index=7}
FIELD_WEIGHTS = {
    'Job Title': 3.0,
    'Role': 2.5,
    'skills': 2.0,
    'Job Description': 1.0,
    'Company': 0.8
}

# Function to combine and weight important text fields for a job posting
def combine_text_fields(job_row):
    """
    Combine text from multiple fields (Job Title, Role, Skills, Job Description, Company)
    with specified weights. Text is cleaned and fields are duplicated according to weight.
    """
    weighted_parts = []
    for field, weight in FIELD_WEIGHTS.items():
        if field in job_row and pd.notnull(job_row[field]):
            text = clean_text(job_row[field])
            # Duplicate the field text int(weight) times
            if weight >= 1:
                weighted_parts.extend([text] * int(weight))
            # Handle fractional part by adding a proportion of the text's words&#8203;:contentReference[oaicite:8]{index=8}
            frac = weight - int(weight)
            if frac > 0 and text:
                words = text.split()
                cutoff = int(len(words) * frac)
                if cutoff > 0:
                    weighted_parts.append(' '.join(words[:cutoff]))
    # Combine all weighted parts into one string
    combined_text = ' '.join(weighted_parts)
    return combined_text.strip()

# Apply the combination to each job posting to create a new combined text column
df_jobs['COMBINED_TEXT'] = df_jobs.apply(combine_text_fields, axis=1)
print("Sample combined text for first job:\n", df_jobs.loc[0, 'COMBINED_TEXT'][:200], "...")


Sample combined text for first job:
 purchasing agent purchasing agent purchasing agent inventory manager inventory manager inventory inventory control demand forecasting supply chain management data analysis warehouse management invento ...


In [9]:
import os
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from scipy.sparse import save_npz, load_npz
from sklearn.metrics.pairwise import cosine_similarity

# --- Text Cleaning Helper ---
def clean_text(text):
    import re
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- TF-IDF Training and Caching ---
def build_and_cache_tfidf(jobs_df, cache_dir="tfidf_cache"):
    os.makedirs(cache_dir, exist_ok=True)
    vec_path = os.path.join(cache_dir, "job_vectors.npz")
    model_path = os.path.join(cache_dir, "vectorizer.pkl")

    vectorizer = TfidfVectorizer(
        stop_words='english',
        max_features=3000,
        min_df=3,
        max_df=0.85,
        ngram_range=(1, 1),
        sublinear_tf=True
    )
    job_matrix = vectorizer.fit_transform(jobs_df['COMBINED_TEXT'])

    save_npz(vec_path, job_matrix)
    joblib.dump(vectorizer, model_path)

    return vectorizer, job_matrix

def load_tfidf_cache(cache_dir="tfidf_cache"):
    vec_path = os.path.join(cache_dir, "job_vectors.npz")
    model_path = os.path.join(cache_dir, "vectorizer.pkl")

    if os.path.exists(vec_path) and os.path.exists(model_path):
        vectorizer = joblib.load(model_path)
        job_matrix = load_npz(vec_path)
        return vectorizer, job_matrix
    return None, None

# --- Main Recommender Function ---
def recommend_jobs_tfidf(resume_text, jobs_df, top_n=5):
    """
    Optimized & cached TF-IDF recommendation system.
    - Reuses or builds cached TF-IDF matrix and vectorizer.
    - Computes cosine similarity via normalized dot product.
    Returns top N indices and similarity scores.
    """
    # Step 1: Load or build cached model
    vectorizer, job_tfidf = load_tfidf_cache()
    if vectorizer is None or job_tfidf is None:
        vectorizer, job_tfidf = build_and_cache_tfidf(jobs_df)

    # Step 2: Normalize job matrix (once for cosine sim)
    job_tfidf = normalize(job_tfidf, axis=1)

    # Step 3: Vectorize and normalize resume
    resume_clean = clean_text(resume_text)
    resume_vec = vectorizer.transform([resume_clean])
    resume_vec = normalize(resume_vec, axis=1)

    # Step 4: Fast cosine similarity using dot product
    sim_scores = resume_vec.dot(job_tfidf.T).toarray().flatten()

    # Step 5: Top N results
    top_idx = sim_scores.argsort()[-top_n:][::-1]
    return top_idx, sim_scores


In [12]:
### TF-IDF Test sample
# After loading or creating your df_jobs
resume_text = extract_resume_text("sampleresume.pdf")  # assume this exists

top_indices, similarities = recommend_jobs_tfidf(resume_text, df_jobs, top_n=5)

# Show results
display_top_jobs(top_indices, similarities, df_jobs)


Top 5 Recommended Jobs:
1. Job at  (Score: 0.028)
2. Job at  (Score: 0.087)
3. Job at  (Score: 0.014)
4. Job at  (Score: 0.021)
5. Job at  (Score: 0.091)
